# Geospatial data processing

Today we're going to look at pulling climate statistics out for suburbs from a shapefile. First let's look at the libraries we're going to use

In [ ]:
import geopandas
import shapely
import pandas
import rasterio
import affine
import numpy as np

We're using `geopandas` to manage the vector data, and `rasterio` to manage the raster data. You can find the documentation for these libraries here:

- [`geopandas` documentation](http://geopandas.org) - uses `shapely` and `pandas` under the hood to manage geometries and tabular attributes respectively.
- [`rasterio` documentation](https://rasterio.readthedocs.io/en/latest/) - uses `numpy` and `affine` under the hood to manage arrays and transformations respectively.

Feel free to have a browse around and see what else these libraries can do for you. There's also a specialized StackExchange (like a subreddit for StackOverflow) on geospatial stuff here: https://gis.stackexchange.com which typically has a higher signal-to-noise ratio if you get stuck.

### Downloading and parsing BoM climate data

I've downloaded the BoM data for Australia from here: http://www.bom.gov.au/jsp/ncc/climate_averages/temperature/index.jsp?maptype=3&period=aut#maps

BoM, in it's own wisdom, uses a Grid text file which has no useful geospatial aspects so we need a seperate function to put it into a useful format. Feel free to dumpster dive if you want to see how to generate your own geotiffs from numpy arrays otherwise just ignore this function and treat it as a black-box conversion service.

In [ ]:
def grid_to_geotiff(filename, return_data=False):
    """ Load data from a Bureau of Meterology 'grid' file and dump out to geotiff
        Output is written to geotiff
        Parameters:
            filename - the file name of the BoM grid file to convert
            return_data - if true, then a Numpy array with the imported
                data is returned when conversion is successful.
    """
    # First six lines are metadata
    # Note: gonna assume the origin is WGS84
    with open(filename) as fhandle:
        meta = {}
        for line in fhandle:
            key, value = line.split()
            key = key.lower()
            meta[key] = value
            if len(meta) == 6:
                break

    # Handle sloppy labelling from BoM
    for idx in ('x', 'y'):
        if meta.get(idx + 'llcorner') is not None:
            meta[idx + 'llcenter'] = meta[idx + 'llcorner']

    # Convert metadata to right format
    type_mapping = {'ncols': int, 'nrows': int,
                    'xllcenter': float, 'yllcenter': float,
                    'cellsize': float, 'nodata_value': float}
    for key, typef in type_mapping.items():
        meta[key] = typef(meta[key])

    # Next lines are info - swap out nodata with nans
    # Last lines are also metadata 'header' but we don't care about that
    data = np.genfromtxt(filename, dtype=np.float64,
                            skip_header=6, skip_footer=18)

    # Check whether we have masked values to deal with
    has_mask = (meta.get('nodata_value') is not None)
    if has_mask:
        has_mask = True
        nodata_mask = (data - meta['nodata_value']) ** 2 < 1e-6

    ## MAKE GEOTIFF
    # Generate the transform for the grid
    aff = affine.Affine.translation(
        meta['xllcenter'] - meta['cellsize'] * 0.5,
        meta['yllcenter'] + (meta['nrows'] - 0.5) * meta['cellsize']
    ) * affine.Affine.scale(meta['cellsize'], -meta['cellsize'])

    # Make metadata for geotiff
    geotiff_meta = {
        'transform': aff,
        'width': meta['ncols'],
        'height': meta['nrows'],
        'nodata': meta['nodata_value'],
        'tiled': 'no',
        'crs': {'init': 'epsg:4326'},  # assuming WGS84
        'driver': 'GTiff',
        'dtype': 'float64',
        'blockxsize': 128,
        'blockysize': 128,
        'count': 1
    }

    # Write out to file
    with rasterio.open(filename + '.geotiff', 'w', **geotiff_meta) as sink:
        sink.write_band(1, data)
        if has_mask:
            sink.write_mask(nodata_mask.astype(bool))

    # If we're returning the data, convert data mask to np.nan
    if return_data:
        if has_mask:
            data[nodata_mask] = np.nan
        return data
    else:
        return filename + '.geotiff'

Now lets use this importer to read the grid file and convert it to a georeferenced GeoTIFF

In [ ]:
data = {
    'spring': grid_to_geotiff('../data/minspr.txt', return_data=True),
    'summer': grid_to_geotiff('../data/minsum.txt', return_data=True),
    'autumn': grid_to_geotiff('../data/minaut.txt', return_data=True),
    'winter': grid_to_geotiff('../data/minwin.txt', return_data=True)
}

We can show a quick-and-dirty plot of Australia's minimum temperatures for each season

In [ ]:
import matplotlib.pyplot as plt

# Set the min and maximum so everything uses the same scale
min_temp = -1
max_temp = 20

# Generate the plots for each season
fig, axes = plt.subplots(1, 4)
for (title, temps), ax in zip(data.items(), axes):
    ax.imshow(temps, vmin=min_temp, vmax=max_temp)
    ax.set_title(title)
    ax.set_axis_off()
fig.set_size_inches(12, 4)

### Getting a suburbs shapefile for Perth

We're going to use the suburbs shapefile to extract portions of these temperature maps for Perth. You can see the info page on `data.gov.au` here: https://data.gov.au/dataset/wa-suburb-locality-boundaries-psma-administrative-boundaries

First, download and extract the shapefile, then you can use `geopandas` to read it just like you'd use `pandas` to read a normal CSV.

In [ ]:
import urllib.request
import zipfile

# Download the file in a temporary folder
url = 'https://data.gov.au/data/dataset/6a0ec945-c880-4882-8a81-4dbcb85e74e5/resource/141fc7bd-c75f-49b5-a116-35250eea68cd/download/wa_locality_polygon_shp.zip'
filepath, _ = urllib.request.urlretrieve(url)
# Unzip it in the data folder
with zipfile.ZipFile(filepath, 'r') as file:
    file.extractall(path='../data')

In [ ]:
suburbs = geopandas.read_file('../data/WA_LOCALITY_POLYGON_SHP.shp')

Let's take a look at what we've got

In [ ]:
suburbs.head()

What we can see is that there is a bunch of rows, each with potentially useful information. The key difference is the `geometry` column - this is a set of shapes which contains a 'geometry' for the suburb - in this case a polygon representing the suburb outline.

In [ ]:
# Quick and dirty map of our suburbs
suburbs.plot(figsize=(12, 12))

Let's look at an individual suburb

In [ ]:
suburb = suburbs[suburbs.LOC_PID == 'WA2863'].iloc[0]  # let's play with Perth's CBD
suburb

In [ ]:
suburb.geometry

This geometry is a regular shapely shape and we can access a whole bunch of analytics from here. For example we can get the boundary:

In [ ]:
suburb.geometry.boundary

We could buffer on this boundary to find locations within a certain distance of the suburb edge

In [ ]:
near_boundary = suburb.geometry.boundary.buffer(0.001)
near_boundary

If we only care about properties inside the suburb we can generate the intersection of the buffer with the suburb

In [ ]:
suburb.geometry.intersection(near_boundary)

We can also get the suburb centroid etc etc

In [ ]:
suburb.geometry.centroid

Basically anything you could think of doing in ArcGIS you can do in Python, with the added advantage of all the machine learning/statistical/data science libraries that you can leverage to do really complex stuff. 

Have a play with the attributes here and also take a look at the [`shapely` documentation for more spatial analysis methods](https://shapely.readthedocs.io/en/stable/manual.html#spatial-analysis-methods)

### Getting temperature data for the suburb

Next we're going to look at using Rasterio to pull out the temperature pixels for each suburb. 

Before we generate the masks we need to make sure that the polygons and the rasters have the same projection. Fortunately `geopandas` makes this easy for us:

In [ ]:
# Check the initial suburb coordinate reference system
suburbs.crs

In [ ]:
# Convert to EPSG 4326 (Web Mercator, aka EPSG:900913)
suburbs.to_crs(epsg=4326, inplace=True)

In [ ]:
# Pull out our suburb again
suburb = suburbs[suburbs.LOC_PID == 'WA2863'].iloc[0]
suburb.geometry  # this will look the same but the numbers are diff

The key piece of functionality we want is in `rasterio.rasterize` (funnily enough my search for copy-paste code [led me to a GitHub issue I contributed to in 2014](https://github.com/mapbox/rasterio/issues/154) - it's nice when past you makes your job easier). Basically we create a mask for the image by burning the polygons into the raster

In [ ]:
from rasterio.features import rasterize

# Generate mask using first image for metadata
mask_value = 1
fill_value = 0
with rasterio.open('../data/minaut.txt.geotiff') as src:
    mask = rasterize([(suburb.geometry.__geo_interface__, mask_value)], 
                     out_shape=src.shape, 
                     transform=src.transform, 
                     all_touched=True, 
                     fill=fill_value).astype(bool)

Note that there might not be many pixels touched by the polygon at this scale, but they're the right ones that we want!

In [ ]:
np.count_nonzero(mask) # see how many pixels we've picked up

Now we can use the mask to select the relevant pixels out of the geotiff to get temperature data for the suburb for each season

In [ ]:
datasets = [
    ('autumn', '../data/minaut.txt.geotiff'),
    ('spring', '../data/minspr.txt.geotiff'),
    ('summer', '../data/minsum.txt.geotiff'),
    ('winter', '../data/minwin.txt.geotiff')
]

results = {}
for key, filename in datasets:
    with rasterio.open(filename) as src:
        results[key] = src.read(1)[mask]
    
results

**Exercise:** Write a function to do this data extraction for all the suburbs in the shapefile with feeding it to a machine learning pipeline in mind. The function should return a pandas dataframe with the LOC_PID of the suburbs, the coordinates of their centroids, and the mean temperatures for each season.